In [3]:
import ee
import geemap

ee.Authenticate()


Successfully saved authorization token.


In [5]:
ee.Initialize()

In [ ]:
def get_kigali_boundary():
    admin2 = ee.FeatureCollection("FAO/GAUL/2015/level2")
    return admin2.filter(ee.Filter.eq('ADM2_NAME', 'Kigali City')).geometry()

# 2. Create farm extent (1ha = 100x100m)
def create_farm_bbox(kigali_geom):
    return kigali_geom.centroid().buffer(50).bounds()

# 3. Load Sentinel-2 with proper cloud masking [2]
def get_s2_image(bbox, start_date='2023-01-01', end_date='2023-12-31'):
    s2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
        .filterBounds(bbox) \
        .filterDate(start_date, end_date) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))

    # Cloud masking function [2]
    def mask_clouds(img):
        qa = img.select('QA60')
        cloud_mask = qa.bitwiseAnd(1 << 10).eq(0)  # Bit 10: clouds
        cirrus_mask = qa.bitwiseAnd(1 << 11).eq(0)  # Bit 11: cirrus
        return img.updateMask(cloud_mask.And(cirrus_mask))

    return s2.map(mask_clouds).median().clip(bbox)

# 4. Compute indices with verified band names [3]
def compute_indices(image):
    return image.addBands([
        image.normalizedDifference(['B8', 'B4']).rename('NDVI'),
        image.normalizedDifference(['B3', 'B8']).rename('NDWI'),
        image.normalizedDifference(['B8', 'B11']).rename('NDBI')
    ])


# Execute workflow
kigali_geom = get_kigali_boundary()
farm_bbox = create_farm_bbox(kigali_geom)
s2_image = get_s2_image(farm_bbox)
processed_image = compute_indices(s2_image)

# Verify band names
print('Available bands:', processed_image.bandNames().getInfo())

# Visualization
Map = geemap.Map()
Map.centerObject(farm_bbox, 14)
Map.addLayer(processed_image, {
             'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}, 'RGB')
Map.addLayer(processed_image.select('NDVI'), {
             'min': 0, 'max': 1, 'palette': ['red', 'yellow', 'green']}, 'NDVI')
Map

EEException: Image.normalizedDifference: No band named 'B8'. Available band names: [].